In [ ]:
import pandas as pd

df = pd.read_excel('~/playground/Expenses/2022.xlsx', sheet_name="USD")
df["Merchant"] = df["Merchant"].str.strip()
df = df.drop(columns=["Notes", "Description"])
df = df[df["Transaction Type"] == "debit"]

df['Date'] =  pd.DatetimeIndex(df.Date)

df['Amount'].sum()

In [ ]:
df = df.assign(parsed=df['Category'].str.split('+')).explode('parsed')
df = df.drop(columns="Category")
#df

In [ ]:
def category(parsed):
    return parsed.split('(').str[0].str.strip()

In [ ]:
def amount(parsed):
    return parsed.split('(').str[1]

In [ ]:
df = df.assign(Category=lambda x: category(x.parsed.str))
df = df.assign(RAmount=lambda x: amount(x.parsed.str))

df['RAmount'] = df['RAmount'].str.split(')').str[0].astype('float')
df['RAmount'].fillna(df['Amount'], inplace=True)
df = df.drop(columns='parsed')

In [ ]:
monthly_report = df.groupby([df.Date.dt.to_period('M'), "Category"])['RAmount'].sum().reset_index(name='Amount').sort_values(by=['Date', 'Amount'], ascending=False)
monthly_report['Amount'].sum()
#monthly_report

In [ ]:
pv_table = pd.pivot_table(monthly_report, index = 'Category', columns = 'Date', values = 'Amount', fill_value=0, dropna=False)
fig = pv_table.plot(kind = 'bar', figsize=(20,10),  yticks=(100, 250, 500, 1000, 1500, 2000, 2500, 5000))

In [ ]:
#fig.get_figure().savefig("monthly_category_report.png", format="png")

In [ ]:
import IPython
HTML = IPython.display.HTML

In [ ]:
category_report = df.groupby(["Category"])['RAmount'].sum().reset_index(name='Amount').sort_values(by=['Amount'], ascending=False)
with pd.option_context("display.max_rows", 1000):
    display(HTML(category_report.to_html(index=False, header=True)))

In [ ]:
annual_monthly_report=category_report.merge(pv_table, on='Category').sort_values(by=['Amount'], ascending=False)
with pd.option_context("display.max_rows", 1000):
    display(HTML(annual_monthly_report.to_html(index=False, header=True)))


In [ ]:
category_report.plot.bar(x='Category', y='Amount', figsize=(20,10),  yticks=(100, 250, 500, 1000, 1500, 2000, 2500, 5000))

In [ ]:
category_report['Amount'].sum()

In [ ]:
merchant_report = df.groupby(["Merchant"])['RAmount'].sum().reset_index(name='Amount').sort_values(by=['Amount'], ascending=False)
with pd.option_context("display.max_rows", 1000):
    display(HTML(merchant_report.to_html(index=False, header=True)))